# NV Saturation measurement

Required modules
- powermeter + powermeterlogic
- laser_toptica + laserlogic 
- fitlogic

In [27]:
import os
import time
import matplotlib.pyplot as plt

from collections import OrderedDict
import datetime

In [28]:
def draw_figure(data):
    powers, counts = data["Powermeter power (W)"], data["Avg. counts"]
    
    plt.style.use(savelogic.mpl_qudihira_style)

    fig, ax = plt.subplots()

    ax.plot(powers, counts, "o--")
    ax.set_xlabel("Powermeter power (W)")
    ax.set_ylabel("Mean counts (kcps)")

    fit_result = fitlogic.make_hyperbolicsaturation_fit(
        powers, counts, estimator=fitlogic.estimate_hyperbolicsaturation
    )
    ax.plot(powers, fit_result.best_fit, '-', label='Hyperbolic Saturation fit')

    ax.legend()
    fig.tight_layout()
    return fig, fit_result
    

def save_data(data_to_save, fit_result, fig, tag=None):
    timestamp = datetime.datetime.now()

    parameters = OrderedDict()
    parameters["Hyperbolic fit"] = fit_result.fit_report()

    data = OrderedDict()
    data["Laser power (W)"] = data_to_save["Laser power (W)"]
    data["Powermeter power (W)"] = data_to_save["Powermeter power (W)"]
    data["UHV objective power (W)"] = data_to_save["UHV objective power (W)"]
    data["RT objective power (W)"] = data_to_save["RT objective power (W)"]
    data["Avg. counts"] = data_to_save["Avg. counts"]

    filepath = savelogic.get_path_for_module(module_name="Saturation")
    
    if tag:
        filelabel = f"{tag}_saturation"
    else:
        filelabel = "saturation"
    
    print(f"Data saved to {os.path.join(filepath, filelabel)}")

    savelogic.save_data(data, filepath=filepath, parameters=parameters, filelabel=filelabel, 
                        timestamp=timestamp, plotfig=fig, delimiter='\t')

In [29]:
count_freq = counterlogic._count_frequency

current_power = laser_toptica.get_power()

laser_powers = np.linspace(5e-3, 100e-3, 20)
counts = np.zeros(len(laser_powers), dtype=int)
powermeter_powers = np.zeros(len(laser_powers))

for idx, laser_power in enumerate(laser_powers):
    try: 
        laserlogic.set_power(laser_power)
    except ValueError:
        time.sleep(1)
        laserlogic.set_power(laser_power)
    
    # Wait for laser to settle
    time.sleep(2)
    
    # Sleep for 5s
    time.sleep(150 / count_freq)
    powermeter_powers[idx] = powermeter.get_power()
    counts[idx] = np.mean(counterlogic.countdata_smoothed[0, -75:])
    
    print(f"[{idx+1}/{len(laser_powers)}] laser power={laser_power * 1e3} mW, powermeter power={powermeter_powers[idx] * 1e6:.2f} uW, counts={counts[idx]}")

laserlogic.set_power(current_power)

data = {
    "Laser power (W)": laser_powers, 
    "Powermeter power (W)": powermeter_powers, 
    "UHV objective power (W)": powermeter_powers * powermeterlogic.calibration_param_uhv, 
    "RT objective power (W)": powermeter_powers * powermeterlogic.calibration_param_rt,
    "Avg. counts": counts
}

[1/20] laser power=5.0 mW, powermeter power=0.00 uW, counts=301
[2/20] laser power=10.0 mW, powermeter power=0.25 uW, counts=3291
[3/20] laser power=15.0 mW, powermeter power=0.45 uW, counts=6862
[4/20] laser power=20.0 mW, powermeter power=1.08 uW, counts=13570
[5/20] laser power=25.0 mW, powermeter power=2.06 uW, counts=22889
[6/20] laser power=30.000000000000004 mW, powermeter power=2.82 uW, counts=27232
[7/20] laser power=34.99999999999999 mW, powermeter power=3.58 uW, counts=33620
[8/20] laser power=40.0 mW, powermeter power=4.21 uW, counts=37948
[9/20] laser power=45.0 mW, powermeter power=5.09 uW, counts=42434
[10/20] laser power=49.99999999999999 mW, powermeter power=5.89 uW, counts=45928
[11/20] laser power=55.0 mW, powermeter power=6.89 uW, counts=49184
[12/20] laser power=60.0 mW, powermeter power=8.30 uW, counts=53528
[13/20] laser power=65.0 mW, powermeter power=9.39 uW, counts=56854
[14/20] laser power=70.0 mW, powermeter power=10.81 uW, counts=60135
[15/20] laser power=7

In [30]:
fig, fit_result = draw_figure(data)
save_data(data, fit_result, fig, tag="NV")

Data saved to Z:\\Data\\20221130_FR0612-F2-2R16_PCB_UHVstage_rotated\2022\12\20221205\Saturation\NV_saturation
